# Data Wrangler Processing Job for Claims Dataset

This notebook executes your Data Wrangler Flow `claims.flow` on the entire dataset using a SageMaker 
Processing Job and will save the processed data to S3.

This notebook saves data from the step `Cast Single Data Type` from `Source: Claims.Csv`. To save from a different step, go to Data Wrangler 
to select a new step to export. 

---

## Contents

1. [Inputs and Outputs](#Inputs-and-Outputs)
1. [Run Processing Job](#Run-Processing-Job)
   1. [Job Configurations](#Job-Configurations)
   1. [Create Processing Job](#Create-Processing-Job)
   1. [Job Status & S3 Output Location](#Job-Status-&-S3-Output-Location)
1. [Optional Next Steps](#(Optional)Next-Steps)
    1. [Load Processed Data into Pandas](#(Optional)-Load-Processed-Data-into-Pandas)
    1. [Train a model with SageMaker](#(Optional)Train-a-model-with-SageMaker)
---

### Loading stored variables
If you ran this notebook before, you may want to re-use the resources you aready created with AWS. Run the cell below to load any prevously created variables. You should see a print-out of the existing variables. If you don't see anything printed then it's probably the first time you are running the notebook! 

In [1]:
import boto3
import sagemaker
region = sagemaker.Session().boto_region_name
boto3.setup_default_session(region_name=region)
boto_session = boto3.Session(region_name=region)
s3_client = boto3.client("s3", region_name=region)
sagemaker_client = boto_session.client("sagemaker")
sess = sagemaker.session.Session(
    boto_session=boto_session, sagemaker_client=sagemaker_client
)

In [2]:
%store -r
%store

Stored variables and their in-db values:
bucket             -> 'sagemaker-us-east-1-875692608981'
prefix             -> 'fraud-detect-demo'


In [3]:
# ======> output_paths
processing_dir = "/opt/ml/processing"

# Inputs and Outputs

The below settings configure the inputs and outputs for the flow export.

<div class="alert alert-info"> 💡 <strong> Configurable Settings </strong>

In <b>Input - Source</b> you can configure the data sources that will be used as input by Data Wrangler

1. For S3 sources, configure the source attribute that points to the input S3 prefixes
2. For all other sources, configure attributes like query_string, database in the source's 
<b>DatasetDefinition</b> object.

If you modify the inputs the provided data must have the same schema and format as the data used in the Flow. 
You should also re-execute the cells in this section if you have modified the settings in any data sources.
</div>

In [4]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.dataset_definition.inputs import AthenaDatasetDefinition, DatasetDefinition, RedshiftDatasetDefinition

data_sources = []

## Input - S3 Source: claims.csv

In [5]:
import uuid
data_sources.append(ProcessingInput(
    source=f"s3://{bucket}/{prefix}/data/raw/claims.csv", # You can override this to point to other dataset on S3
    destination=f"{processing_dir}/claims",
    input_name="claims",
    s3_data_type="S3Prefix",
    s3_input_mode="File",
    s3_data_distribution_type="FullyReplicated"
))
# print(f"Input source name: {input_name}")

## Output: S3 settings

<div class="alert alert-info"> 💡 <strong> Configurable Settings </strong>

1. <b>bucket</b>: you can configure the S3 bucket where Data Wrangler will save the output. The default bucket from 
the SageMaker notebook session is used. 
2. <b>flow_export_id</b>: A randomly generated export id. The export id must be unique to ensure the results do not 
conflict with other flow exports 
3. <b>s3_ouput_prefix</b>:  you can configure the directory name in your bucket where your data will be saved.
</div>

In [6]:
import time

flow_export_id = f"{time.strftime('%d-%H-%M-%S', time.gmtime())}-{str(uuid.uuid4())[:8]}"
flow_export_name = f"flow-{flow_export_id}"
print(f"Flow export name: {flow_export_name}")

Flow export name: flow-12-13-38-55-035c38b2


Below are the inputs required by the SageMaker Python SDK to launch a processing job.

In [7]:
import json
# name of the flow file which should exist in the current notebook working directory
flow_file_name = "flows/claims.flow"

# Load .flow file from current notebook working directory 
!echo "Loading flow file from current notebook working directory: $PWD"

with open(flow_file_name) as f:
    flow = json.load(f)

# Output name is auto-generated from the select node's ID + output name from the flow file.
output_name = (f"{flow['nodes'][-1]['node_id']}.{flow['nodes'][-1]['outputs'][0]['name']}")
print(f"Output name: {output_name}")

s3_output_prefix = f"export-{flow_export_name}/output"
s3_output_path = f"s3://{bucket}/{prefix}/flow/output/{s3_output_prefix}"
print(f"Flow S3 export result path: {s3_output_path}")

processing_job_output = ProcessingOutput(
    output_name=output_name,
    source=f"{processing_dir}/output",
    destination=s3_output_path,
    s3_upload_mode="EndOfJob"
)

Loading flow file from current notebook working directory: /root/mlt
Output name: 62d710d9-a288-4004-b960-6cf452c0380c.default
Flow S3 export result path: s3://sagemaker-us-east-1-875692608981/fraud-detect-demo/flow/output/export-flow-12-13-38-55-035c38b2/output


## Upload Flow to S3

To use the Data Wrangler as an input to the processing job,  first upload your flow file to Amazon S3.

In [8]:
# Upload flow to S3
s3_client = boto3.client("s3")
s3_client.upload_file(flow_file_name, bucket, f"{prefix}/data_wrangler_flows/{flow_export_name}.flow", ExtraArgs={"ServerSideEncryption": "aws:kms"})

flow_s3_uri = f"s3://{bucket}/{prefix}/data_wrangler_flows/{flow_export_name}.flow"

print(f"Data Wrangler flow {flow_file_name} uploaded to {flow_s3_uri}")

Data Wrangler flow flows/claims.flow uploaded to s3://sagemaker-us-east-1-875692608981/fraud-detect-demo/data_wrangler_flows/flow-12-13-38-55-035c38b2.flow


The Data Wrangler Flow is also provided to the Processing Job as an input source which we configure below.

In [9]:
## Input - Flow: claims.flow
flow_input = ProcessingInput(
    source=flow_s3_uri,
    destination=f"{processing_dir}/flow",
    input_name="flow",
    s3_data_type="S3Prefix",
    s3_input_mode="File",
    s3_data_distribution_type="FullyReplicated"
)

# Run Processing Job 
## Job Configurations

<div class="alert alert-info"> 💡 <strong> Configurable Settings </strong>

You can configure the following settings for Processing Jobs. If you change any configurations you will 
need to re-execute this and all cells below it by selecting the Run menu above and click 
<b>Run Selected Cells and All Below</b>

1. IAM role for executing the processing job. 
2. A unique name of the processing job. Give a unique name every time you re-execute processing jobs
3. Data Wrangler Container URL.
4. Instance count, instance type and storage volume size in GB.
5. Content type for each output. Data Wrangler supports CSV as default and Parquet.
6. Network Isolation settings
7. KMS key to encrypt output data
</div>

In [10]:
from sagemaker import image_uris
# IAM role for executing the processing job.
iam_role = sagemaker.get_execution_role()

# Unique processing job name. Give a unique name every time you re-execute processing jobs
processing_job_name = f"data-wrangler-flow-processing-{flow_export_id}"
print(f"Processing Job Name: {processing_job_name}")
# Data Wrangler Container URL.
container_uri = image_uris.retrieve(framework='data-wrangler',region=region)
print(f"Container uri: {container_uri}")

# Processing Job Instance count and instance type.
instance_count = 2
instance_type = "ml.m5.4xlarge"

# Size in GB of the EBS volume to use for storing data during processing
volume_size_in_gb = 30

# Content type for each output. Data Wrangler supports CSV as default and Parquet.
output_content_type = "CSV"

# Network Isolation mode; default is off
enable_network_isolation = False

# Output configuration used as processing job container arguments 
output_config = {
    output_name: {
        "content_type": output_content_type
    }
}

# KMS key for per object encryption; default is None
kms_key = None

Processing Job Name: data-wrangler-flow-processing-12-13-38-55-035c38b2
Container uri: 663277389841.dkr.ecr.us-east-1.amazonaws.com/sagemaker-data-wrangler-container:1.x


## Create Processing Job

To launch a Processing Job, you will use the SageMaker Python SDK to create a Processor function.

In [11]:
from sagemaker.processing import Processor
from sagemaker.network import NetworkConfig

processor = Processor(
    role=iam_role,
    image_uri=container_uri,
    instance_count=instance_count,
    instance_type=instance_type,
    volume_size_in_gb=volume_size_in_gb,
    network_config=NetworkConfig(enable_network_isolation=enable_network_isolation),
    sagemaker_session=sess,
    output_kms_key=kms_key
)

# Start Job
processor.run(
    inputs=[flow_input] + data_sources, 
    outputs=[processing_job_output],
    arguments=[f"--output-config '{json.dumps(output_config)}'"],
    wait=False,
    logs=False,
    job_name=processing_job_name
)


Job Name:  data-wrangler-flow-processing-12-13-38-55-035c38b2
Inputs:  [{'InputName': 'flow', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-875692608981/fraud-detect-demo/data_wrangler_flows/flow-12-13-38-55-035c38b2.flow', 'LocalPath': '/opt/ml/processing/flow', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'claims', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-875692608981/fraud-detect-demo/data/raw/claims.csv', 'LocalPath': '/opt/ml/processing/claims', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': '62d710d9-a288-4004-b960-6cf452c0380c.default', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-875692608981/fraud-detect-demo/flow/output/export-flow-12-13-38-55-035c38b2/output', 'LocalPath': '/opt/ml/processing/output', 'S3Upl

## Job Status & S3 Output Location

Below you wait for processing job to finish. If it finishes successfully, the raw parameters used by the 
Processing Job will be printed

In [12]:
s3_job_results_path = f"s3://{bucket}/{prefix}/flow/{s3_output_prefix}/{processing_job_name}"
print(f"Job results are saved to S3 path: {s3_job_results_path}")

job_result = sess.wait_for_processing_job(processing_job_name)
job_result

Job results are saved to S3 path: s3://sagemaker-us-east-1-875692608981/fraud-detect-demo/flow/export-flow-12-13-38-55-035c38b2/output/data-wrangler-flow-processing-12-13-38-55-035c38b2
.........................................................................!

{'ProcessingInputs': [{'InputName': 'flow',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-875692608981/fraud-detect-demo/data_wrangler_flows/flow-12-13-38-55-035c38b2.flow',
    'LocalPath': '/opt/ml/processing/flow',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'claims',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-875692608981/fraud-detect-demo/data/raw/claims.csv',
    'LocalPath': '/opt/ml/processing/claims',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': '62d710d9-a288-4004-b960-6cf452c0380c.default',
    'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-875692608981/fraud-detect-demo/flow/output/export-flow-12-13-38-55-035c38b2/output',
     'LocalPath': '/opt/ml/proce

<a id='aud-datasets'></a>

## DataSets and Feature Types
[overview](#all-up-overview)
----

In [13]:
claims_dtypes = {
    "policy_id": int,
    "incident_severity": int,
    "num_vehicles_involved": int,
    "num_injuries": int,
    "num_witnesses": int,
    "police_report_available": int,
    "injury_claim": float,
    "vehicle_claim": float,
    "total_claim_amount": float,
    "incident_month": int,
    "incident_day": int,
    "incident_dow": int,
    "incident_hour": int,
    "fraud": int,
    "driver_relationship_self": int,
    "driver_relationship_na": int,
    "driver_relationship_spouse": int,
    "driver_relationship_child": int,
    "driver_relationship_other": int,
    "incident_type_collision": int,
    "incident_type_breakin": int,
    "incident_type_theft": int,
    "collision_type_front": int,
    "collision_type_rear": int,
    "collision_type_side": int,
    "collision_type_na": int,
    "authorities_contacted_police": int,
    "authorities_contacted_none": int,
    "authorities_contacted_fire": int,
    "authorities_contacted_ambulance": int,
    "event_time": float,
}

### Load Processed Data into Pandas

We use the [AWS Data Wrangler library](https://github.com/awslabs/aws-data-wrangler) to load the exported 
dataset into a Pandas dataframe.

In [14]:
import awswrangler as wr

In [15]:
# ======> This is your DataFlow output path if you decide to redo the work in DataFlow on your own
if output_content_type.upper() == "CSV":
    claims_preprocessed = wr.s3.read_csv(
        path=s3_output_path, dataset=True, dtype=claims_dtypes
    )
else:
    print(f"Unexpected output content type {output_content_type}")

%store claims_preprocessed
claims_preprocessed

Stored 'claims_preprocessed' (DataFrame)


,policy_id,incident_severity,num_vehicles_involved,num_injuries,num_witnesses,police_report_available,injury_claim,vehicle_claim,total_claim_amount,incident_month,...,incident_type_theft,collision_type_front,collision_type_rear,collision_type_side,collision_type_na,authorities_contacted_police,authorities_contacted_none,authorities_contacted_ambulance,authorities_contacted_fire,event_time
0,1,0,2,0,0,0,71600.0,8913.0,80513.0,3,...,0,1,0,0,0,0,1,0,0,1.636725e+09
1,2,2,3,4,0,1,6400.0,19746.0,26146.0,12,...,0,0,1,0,0,1,0,0,0,1.636725e+09
2,3,0,2,0,1,1,10400.0,11652.0,22052.0,12,...,0,1,0,0,0,1,0,0,0,1.636725e+09
3,4,0,2,0,0,0,104700.0,11260.0,115960.0,12,...,0,0,0,1,0,0,1,0,0,1.636725e+09
4,5,1,2,1,0,0,3400.0,27987.0,31387.0,5,...,0,0,0,1,0,1,0,0,0,1.636725e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,0,2,0,3,0,15700.0,1494.0,17194.0,8,...,0,1,0,0,0,0,1,0,0,1.636725e+09
4996,4997,0,2,0,1,0,34900.0,14837.0,49737.0,1,...,0,0,0,1,0,0,1,0,0,1.636725e+09
4997,4998,0,1,0,0,0,11700.0,12421.0,24121.0,6,...,0,1,0,0,0,0,1,0,0,1.636725e+09
4998,4999,0,3,0,2,1,14000.0,10991.0,24991.0,3,...,0,0,1,0,0,1,0,0,0,1.636725e+09


We now have a set of Pandas DataFrames that contain the customer and claim data, with the correct data types. When Dat Wrangler encodes a feature as one-hot-encoded feature, it will default to float data types for those resulting features (one feature --> many columns for the one hot encoding). 

<font color ='red'> Note: </font> the reason for explicitly converting the data types for categorical features generated by Data Wrangler, is to ensure they are of type integer so that Clarify will treat them as categorical variables. 